In [3]:
# Importing the required libraries
import os
import random
import pandas as pd

import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
from ultralytics import YOLO

AttributeError: module 'torch.ao.nn.quantized.modules' has no attribute 'conv'

In [ ]:
# Define the paths to the images and labels directories
train_images = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/train/images'
train_labels = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/train/labels'

val_images = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/valid/images'
val_labels = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/valid/labels'

test_images = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/test/images'
test_labels = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/test/labels'

In [ ]:
# Define the path to the yaml data file
yaml_path = 'D:/GIT HUB/Underwater-object-detection-using-yolov8/codes/data.yaml'

In [ ]:
# Define the labels
classes = ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']
Idx2Label = {idx: label for idx, label in enumerate(classes)}
Label2Index = {label: idx for idx, label in Idx2Label.items()}
print('Index to Label Mapping:', Idx2Label)
print('Label to Index Mapping:', Label2Index)

In [ ]:
def visualize_image_with_annotation_bboxes(image_dir, label_dir):
    # Get list of all the image files in the directory
    image_files = sorted(os.listdir(image_dir))
    
    # Choose 10 random image files from the list
    sample_image_files = random.sample(image_files, 12)
    
    # Set up the plot
    fig, axs = plt.subplots(4, 3, figsize=(15, 20))
    
    # Loop over the random images and plot the bounding boxes
    for i, image_file in enumerate(sample_image_files):
        row = i // 3
        col = i % 3
        
        # Load the image
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load the labels for this image
        label_path = os.path.join(label_dir, image_file[:-4] + '.txt')
        f = open(label_path, 'r')
        
        # Loop over the labels and plot the bounding boxes
        for label in f:
            class_id, x_center, y_center, width, height = map(float, label.split())
            h, w, _ = image.shape
            x_min = int((x_center - width/2) * w)
            y_min = int((y_center - height/2) * h)
            x_max = int((x_center + width/2) * w)
            y_max = int((y_center + height/2) * h)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(image, Idx2Label[int(class_id)], (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    
        axs[row, col].imshow(image)
        axs[row, col].axis('off')

    #plt.show()
    plt.savefig('output_image_grid.png')  # Save the figure as an image file
    plt.close()  # Close the plot to avoid placeholder output
    print("Figure saved as 'output_image_grid.png'")

# Visualize 12 sample images with bounding boxes
visualize_image_with_annotation_bboxes(train_images, train_labels)

In [ ]:
# Load a YOLOv8 model, for example, the nano version for fast training
model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt', 'yolov8m.pt', etc. for different sizes

# Train the model
results = model.train(
    data="D:/GIT HUB/Underwater-object-detection-using-yolov8/codes/underwaterobject.yaml",  # Path to the dataset YAML file
    epochs=100,                                 # Number of training epochs (adjust as needed)
    imgsz=640,                                 # Image size for training (can be adjusted)
    lr0=0.01,                                  # Initial learning rate
    lrf=1,                                   # Final learning rate (fraction of initial LR)
    batch=8,                                   # Batch size (adjust based on your CPU/GPU memory)
    device='cpu',                              # Using 'cpu' since CUDA is not available
    name='underwater_detection'                # Folder name for saving the model and results
)

# Extract the mAP score
maP_50 = results.metrics['metrics/mAP50(B)']   # mAP@0.5 score
print(f"mAP@0.5: {maP_50:.4f}")

NameError: name 'YOLO' is not defined